In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture   
%pwd


In [3]:
%%capture
cd ..

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import psycopg2 as ps2
import os
import pandas as pd
import numpy as np
import csv
from src.funcs.utils import *

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# pull in db credentials from .env file
db_name=os.getenv('DB_NAME')
db_user=os.getenv('DB_USER')
db_host=os.getenv('DB_HOST_IP')
db_pwd=os.getenv('DB_PWD')

In [7]:
# conn.close()
# conn.rollback()

In [8]:
# define db session
conn = ps2.connect(
    host = db_host,
    database = db_name,
    user = db_user,
    password = db_pwd)

## Relationships

### Query for data

### Directional

In [ ]:
sql="SELECT \
control.id, inkey[1], controltype,  string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), \
num_refs, outkey[1] , reference.id, string_agg(distinct(biomarkertype), ', ') , string_agg(distinct(celllinename), ', '), \
string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), string_agg(distinct(organ), ', '), \
string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', '),  \
string_agg(distinct(nct_id), ', '),  string_agg(distinct(phase), ', ') \
FROM resnet18.control, resnet18.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null \
GROUP BY control.id, inkey[1], controltype, num_refs, outkey[1], reference.id"

In [ ]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/directional_rels_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer = csv.writer(f, delimiter='|')
        for record in cur.fetchall():
            csv_writer.writerow(record)  

### Bi-derectional

In [ ]:
sql="SELECT \
control.id, inkey[1], inoutkey, controltype, relationship, string_agg(distinct(effect), ', '), string_agg(distinct(mechanism), ', '), num_refs, outkey[1], \
reference.id, string_agg(distinct(biomarkertype), ', '), string_agg(distinct(celllinename), ', '), string_agg(distinct(celltype), ', '), string_agg(distinct(changetype), ', '), \
string_agg(distinct(organ), ', '), string_agg(distinct(organism), ', '), string_agg(distinct(quantitativetype), ', '), string_agg(distinct(tissue), ', ') \
FROM resnet18.control, resnet18.reference \
WHERE control.id = reference.id and inkey[1] is null and outkey[1] is null \
GROUP BY control.id, controltype, reference.id"

In [ ]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/processed/bidirectional_rels_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

### Attributes

In [ ]:
sql="SELECT  \
id, inkey[1], attributes, relationship, outkey[1] from resnet18.control \
WHERE (control.id = control.attributes and inkey[1] is not null and outkey[1] is not null)";

In [ ]:
%%time
with conn.cursor() as cur:
    with open('./data/processed/attribute_rels_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

## Process Data

### Bi-directional

In [9]:
%%time

df_birect=pd.read_csv('./data/processed/bidirectional_rels_raw.txt', sep='|', header=None, encoding='utf-8')
df_birect.info(); df_birect.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5460397 entries, 0 to 5460396
Data columns (total 18 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       int64  
 1   1       float64
 2   2       object 
 3   3       object 
 4   4       float64
 5   5       object 
 6   6       object 
 7   7       int64  
 8   8       float64
 9   9       int64  
 10  10      float64
 11  11      object 
 12  12      object 
 13  13      object 
 14  14      object 
 15  15      object 
 16  16      float64
 17  17      object 
dtypes: float64(5), int64(3), object(10)
memory usage: 749.9+ MB
CPU times: user 9.69 s, sys: 3.67 s, total: 13.4 s
Wall time: 22.9 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9223367021478908826,NaN,"[5747547533469447186, 6031869671835980567]",FunctionalAssociation,NaN,NaN,NaN,1,NaN,-9223367021478908826,NaN,NaN,NaN,NaN,liver,NaN,NaN,NaN
1,-9223365481314032826,NaN,"[5445426109658919917, -2285403010002775161]",FunctionalAssociation,NaN,NaN,NaN,1,NaN,-9223365481314032826,NaN,NaN,NaN,NaN,leg,NaN,NaN,NaN
2,-9223361153568711706,NaN,"[3492638404650443242, -6868785757227974641]",FunctionalAssociation,NaN,NaN,NaN,1,NaN,-9223361153568711706,NaN,NaN,NaN,NaN,brain,Mus musculus (mouse),NaN,NaN
3,-9223357200775258657,NaN,"[1596328558605610184, -8708891732207432195]",CellExpression,NaN,NaN,NaN,8,NaN,-9223357200775258657,NaN,MDA-MB-231,NaN,NaN,breast,"Homo sapiens (human), Oryctolagus cuniculus (r...",NaN,serum
4,-9223356961959519979,NaN,"[4541556823765750071, 3096071376414863663]",CellExpression,NaN,NaN,NaN,1,NaN,-9223356961959519979,NaN,A2058,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_birect.columns=['msrc_id', ':START_ID', 'inOutkey', 'type:TYPE', 'relationship', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', 'biomarkertype', 'celllinename', 'celltype',
      'changetype', 'organ', 'organism', 'quantitativetype', 'tissue']

df_birect.head()

,msrc_id,:START_ID,inOutkey,type:TYPE,relationship,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,-9223367021478908826,NaN,"[5747547533469447186, 6031869671835980567]",FunctionalAssociation,NaN,NaN,NaN,1,NaN,-9223367021478908826,NaN,NaN,NaN,NaN,liver,NaN,NaN,NaN
1,-9223365481314032826,NaN,"[5445426109658919917, -2285403010002775161]",FunctionalAssociation,NaN,NaN,NaN,1,NaN,-9223365481314032826,NaN,NaN,NaN,NaN,leg,NaN,NaN,NaN
2,-9223361153568711706,NaN,"[3492638404650443242, -6868785757227974641]",FunctionalAssociation,NaN,NaN,NaN,1,NaN,-9223361153568711706,NaN,NaN,NaN,NaN,brain,Mus musculus (mouse),NaN,NaN
3,-9223357200775258657,NaN,"[1596328558605610184, -8708891732207432195]",CellExpression,NaN,NaN,NaN,8,NaN,-9223357200775258657,NaN,MDA-MB-231,NaN,NaN,breast,"Homo sapiens (human), Oryctolagus cuniculus (r...",NaN,serum
4,-9223356961959519979,NaN,"[4541556823765750071, 3096071376414863663]",CellExpression,NaN,NaN,NaN,1,NaN,-9223356961959519979,NaN,A2058,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# create separate columns for inOutkeys
first_ids, second_ids = inOutkeys_to_lists(df_birect)

len(first_ids); len(second_ids)
first_ids[:5]; second_ids[:5]

5460397

5460397

['5747547533469447186',
 '5445426109658919917',
 '3492638404650443242',
 '1596328558605610184',
 '4541556823765750071']

['6031869671835980567',
 '-2285403010002775161',
 '-6868785757227974641',
 '-8708891732207432195',
 '3096071376414863663']

In [12]:
df_birect_1=df_birect.copy()
df_birect_1.drop(columns=['inOutkey', 'id2', 'relationship'], inplace=True)

In [13]:
df_birect_1[':START_ID']=first_ids
df_birect_1[':END_ID']=second_ids
df_birect_1.info(); df_birect_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5460397 entries, 0 to 5460396
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   msrc_id           int64  
 1   :START_ID         object 
 2   type:TYPE         object 
 3   effect            object 
 4   mechanism         object 
 5   ref_count:int     int64  
 6   :END_ID           object 
 7   biomarkertype     float64
 8   celllinename      object 
 9   celltype          object 
 10  changetype        object 
 11  organ             object 
 12  organism          object 
 13  quantitativetype  float64
 14  tissue            object 
dtypes: float64(2), int64(2), object(11)
memory usage: 624.9+ MB


,msrc_id,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,-9223367021478908826,5747547533469447186,FunctionalAssociation,NaN,NaN,1,6031869671835980567,NaN,NaN,NaN,NaN,liver,NaN,NaN,NaN
1,-9223365481314032826,5445426109658919917,FunctionalAssociation,NaN,NaN,1,-2285403010002775161,NaN,NaN,NaN,NaN,leg,NaN,NaN,NaN
2,-9223361153568711706,3492638404650443242,FunctionalAssociation,NaN,NaN,1,-6868785757227974641,NaN,NaN,NaN,NaN,brain,Mus musculus (mouse),NaN,NaN
3,-9223357200775258657,1596328558605610184,CellExpression,NaN,NaN,8,-8708891732207432195,NaN,MDA-MB-231,NaN,NaN,breast,"Homo sapiens (human), Oryctolagus cuniculus (r...",NaN,serum
4,-9223356961959519979,4541556823765750071,CellExpression,NaN,NaN,1,3096071376414863663,NaN,A2058,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_birect_1[':START_ID']=df_birect_1[':START_ID'].astype('int64')
df_birect_1[':END_ID']=df_birect_1[':END_ID'].astype('int64')
df_birect_1['ref_count:int']=df_birect_1['ref_count:int'].astype('int16')

df_birect_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5460397 entries, 0 to 5460396
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   msrc_id           int64  
 1   :START_ID         int64  
 2   type:TYPE         object 
 3   effect            object 
 4   mechanism         object 
 5   ref_count:int     int16  
 6   :END_ID           int64  
 7   biomarkertype     float64
 8   celllinename      object 
 9   celltype          object 
 10  changetype        object 
 11  organ             object 
 12  organism          object 
 13  quantitativetype  float64
 14  tissue            object 
dtypes: float64(2), int16(1), int64(3), object(9)
memory usage: 593.6+ MB


In [15]:
# convert to category to save memory
df_birect_1=convert_object_to_category(df_birect_1)
df_birect_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5460397 entries, 0 to 5460396
Data columns (total 15 columns):
 #   Column            Dtype   
---  ------            -----   
 0   msrc_id           int64   
 1   :START_ID         int64   
 2   type:TYPE         category
 3   effect            category
 4   mechanism         category
 5   ref_count:int     int16   
 6   :END_ID           int64   
 7   biomarkertype     float64 
 8   celllinename      category
 9   celltype          category
 10  changetype        category
 11  organ             category
 12  organism          category
 13  quantitativetype  float64 
 14  tissue            category
dtypes: category(9), float64(2), int16(1), int64(3)
memory usage: 353.9 MB


In [ ]:
df_birect_1.to_pickle('./data/processed/bidirectional_rels_procd.pkl')

### Directional

In [19]:
%%time

df_direct=pd.read_csv('./data/processed/directional_rels_raw.txt', sep='|', encoding='utf-8', header=None)

df_direct.info(); df_direct.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11151487 entries, 0 to 11151486
Data columns (total 18 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       int64 
 2   2       object
 3   3       object
 4   4       object
 5   5       int64 
 6   6       int64 
 7   7       int64 
 8   8       object
 9   9       object
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
 15  15      object
 16  16      object
 17  17      object
dtypes: int64(5), object(13)
memory usage: 1.5+ GB
CPU times: user 16.5 s, sys: 8.8 s, total: 25.3 s
Wall time: 56 s


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-9223368466750536065,2590362169334839073,Regulation,NaN,NaN,2,3888856228337743280,-9223368466750536065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9223366135359139662,6434163658456377940,MolSynthesis,NaN,NaN,3,4596447325930325903,-9223366135359139662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-9223365986387833915,-4129340824583713637,Regulation,NaN,NaN,1,-2983484129356197270,-9223365986387833915,NaN,NaN,NaN,NaN,intestine,Rattus norvegicus (rat),NaN,NaN,NaN,NaN
3,-9223365556180770043,7111680438658927661,Regulation,NaN,NaN,1,-3121142714757276759,-9223365556180770043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,serum,NaN,NaN
4,-9223365466130767478,-8447327320654612427,Regulation,NaN,NaN,9,-1646880163763186214,-9223365466130767478,NaN,TOV-112D,"B-cell, endothelial cell",NaN,"peripheral nervous system, synovial membrane",NaN,NaN,NaN,NaN,NaN


In [20]:
df_direct.columns=['msrc_id', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [21]:
df_direct.info(); df_direct.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11151487 entries, 0 to 11151486
Data columns (total 18 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   msrc_id           int64 
 1   :START_ID         int64 
 2   type:TYPE         object
 3   effect            object
 4   mechanism         object
 5   ref_count:int     int64 
 6   :END_ID           int64 
 7   id2               int64 
 8   biomarkertype     object
 9   celllinename      object
 10  celltype          object
 11  changetype        object
 12  organ             object
 13  organism          object
 14  quantitativetype  object
 15  tissue            object
 16  nct_id            object
 17  phase             object
dtypes: int64(5), object(13)
memory usage: 1.5+ GB


,msrc_id,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223368466750536065,2590362169334839073,Regulation,NaN,NaN,2,3888856228337743280,-9223368466750536065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9223366135359139662,6434163658456377940,MolSynthesis,NaN,NaN,3,4596447325930325903,-9223366135359139662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-9223365986387833915,-4129340824583713637,Regulation,NaN,NaN,1,-2983484129356197270,-9223365986387833915,NaN,NaN,NaN,NaN,intestine,Rattus norvegicus (rat),NaN,NaN,NaN,NaN
3,-9223365556180770043,7111680438658927661,Regulation,NaN,NaN,1,-3121142714757276759,-9223365556180770043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,serum,NaN,NaN
4,-9223365466130767478,-8447327320654612427,Regulation,NaN,NaN,9,-1646880163763186214,-9223365466130767478,NaN,TOV-112D,"B-cell, endothelial cell",NaN,"peripheral nervous system, synovial membrane",NaN,NaN,NaN,NaN,NaN


In [22]:
df_direct=df_direct.drop(columns=['id2'])

df_direct.head()

,msrc_id,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223368466750536065,2590362169334839073,Regulation,NaN,NaN,2,3888856228337743280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9223366135359139662,6434163658456377940,MolSynthesis,NaN,NaN,3,4596447325930325903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-9223365986387833915,-4129340824583713637,Regulation,NaN,NaN,1,-2983484129356197270,NaN,NaN,NaN,NaN,intestine,Rattus norvegicus (rat),NaN,NaN,NaN,NaN
3,-9223365556180770043,7111680438658927661,Regulation,NaN,NaN,1,-3121142714757276759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,serum,NaN,NaN
4,-9223365466130767478,-8447327320654612427,Regulation,NaN,NaN,9,-1646880163763186214,NaN,TOV-112D,"B-cell, endothelial cell",NaN,"peripheral nervous system, synovial membrane",NaN,NaN,NaN,NaN,NaN


In [23]:
df_direct['phase']=df_direct['phase'].fillna('None')

In [24]:
# convert to category dtype to save memory
df_direct=convert_object_to_category(df_direct)
    
df_direct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11151487 entries, 0 to 11151486
Data columns (total 17 columns):
 #   Column            Dtype   
---  ------            -----   
 0   msrc_id           int64   
 1   :START_ID         int64   
 2   type:TYPE         category
 3   effect            category
 4   mechanism         category
 5   ref_count:int     int64   
 6   :END_ID           int64   
 7   biomarkertype     category
 8   celllinename      category
 9   celltype          category
 10  changetype        category
 11  organ             category
 12  organism          category
 13  quantitativetype  category
 14  tissue            category
 15  nct_id            category
 16  phase             category
dtypes: category(13), int64(4)
memory usage: 719.2 MB


In [25]:
df_direct['ref_count:int']=df_direct['ref_count:int'].astype('int16')

In [26]:
df_direct.isnull().sum()

msrc_id             0
:START_ID           0
type:TYPE           0
effect              0
mechanism           0
ref_count:int       0
:END_ID             0
biomarkertype       0
celllinename        0
celltype            0
changetype          0
organ               0
organism            0
quantitativetype    0
tissue              0
nct_id              0
phase               0
dtype: int64

In [27]:
df_direct['type:TYPE'].value_counts()

Regulation               6948309
Expression               1064239
DirectRegulation          792230
QuantitativeChange        561392
GeneticChange             549985
MolTransport              298336
MolSynthesis              193659
Biomarker                 174491
StateChange               164359
ClinicalTrial             140983
ProtModification           82540
miRNAEffect                72018
ChemicalReaction           63542
PromoterBinding            45403
FunctionalAssociation          1
Name: type:TYPE, dtype: int64

In [ ]:
df_direct.to_pickle('./data/processed/directional_rels_procd.pkl')

### Attributes

In [28]:
%%time

df_att=pd.read_csv('./data/processed/attribute_rels_raw.txt', sep='|', header=None, encoding='utf-8')

df_att.info(); df_att.head()

<timed exec>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12264718 entries, 0 to 12264717
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       int64 
 1   1       int64 
 2   2       int64 
 3   3       object
 4   4       int64 
dtypes: int64(4), object(1)
memory usage: 467.9+ MB
CPU times: user 6.14 s, sys: 1.89 s, total: 8.03 s
Wall time: 21 s


,0,1,2,3,4
0,-5796863195923429565,8750092258094575716,-5796863195923429565,is-a,-8777874296820158997
1,293546921283257022,-6517823221527382077,293546921283257022,NaN,-5326345662687474979
2,296455145851088043,8038933735751685804,296455145851088043,NaN,1547222595974281177
3,304327506551372941,-8419665601423615153,304327506551372941,NaN,-8526474292922723918
4,304546577606020441,6856256389387051829,304546577606020441,NaN,8932347568585589874


In [29]:
df_att.columns=['msrc_id', ':START_ID', 'id2', 'type:TYPE', ':END_ID']

df_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12264718 entries, 0 to 12264717
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   msrc_id    int64 
 1   :START_ID  int64 
 2   id2        int64 
 3   type:TYPE  object
 4   :END_ID    int64 
dtypes: int64(4), object(1)
memory usage: 467.9+ MB


In [30]:
df_att['type:TYPE'].value_counts()

is-a             1109167
part-of             3336
derivative-of        728
Name: type:TYPE, dtype: int64

In [31]:
df_att['type:TYPE'] = df_att['type:TYPE'].str.replace('-', '_')
df_att['type:TYPE'].value_counts()

is_a             1109167
part_of             3336
derivative_of        728
Name: type:TYPE, dtype: int64

In [ ]:
df_att=df_att.dropna(how='any')

df_att.reset_index(drop=True, inplace=True)
df_att. info(); df_att.head()

In [ ]:
df_att=df_att.drop(columns=['id2'])

df_att.info(); df_att.head()

In [ ]:
# remove any unwanted white space
for col in list(df_att.columns):
    df_att[col]=df_att[col].apply(lambda x: str(x).strip())

In [ ]:
df_att[':START_ID'] = pd.to_numeric(df_att[':START_ID'], errors='coerce').astype('int64')
df_att[':END_ID'] = pd.to_numeric(df_att[':END_ID'], errors='coerce').astype('int64')
df_att['type:TYPE']=df_att['type:TYPE'].astype('category')


In [ ]:
df_att.head()

In [ ]:
df_att['type:TYPE'].value_counts()

In [ ]:
df_att.to_pickle('./data/processed/attribute_rels_procd.pkl')

### Concatenate Relationships

In [ ]:
df_directional=pd.read_pickle('./data/processed/directional_rels_procd.pkl')
df_directional.head(); df_directional.info()

In [ ]:
df_bidirect=pd.read_pickle('./data/processed/bidirectional_rels_procd.pkl')
df_bidirect.head(); df_bidirect.info()

In [ ]:
df_att=pd.read_pickle('./data/processed/attribute_rels_procd.pkl')
df_att.head(); df_att.info()

In [ ]:
%%time

df_concat=pd.concat([df_directional, df_bidirect, df_att])

In [ ]:
df_concat.info(); df_concat.head()

In [ ]:
for col in df_concat.select_dtypes(include=['category']).columns:
    df_concat[col] = df_concat[col].astype('object')

In [ ]:
df_concat.info()

In [ ]:
df_concat.isnull().sum()

In [ ]:
df_concat['ref_count:int'].fillna(0, inplace=True)

In [ ]:
df_concat=df_concat.fillna('None')

In [ ]:
df_concat=df_concat.replace('nan', 'None')

In [ ]:
df_concat.head(); df_concat.info()

In [ ]:
# set ref count dtype
df_concat['ref_count:int']=df_concat['ref_count:int'].astype('int16')
df_concat['msrc_id']=df_concat['msrc_id'].astype('int64')

In [ ]:
# change object to category to save memory
df_concat=convert_object_to_category(df_concat)

df_concat.info()

In [ ]:
df_concat.reset_index(drop=True, inplace=True)
df_concat.info()

In [ ]:
df_concat.drop_duplicates(inplace=True)
df_concat.reset_index(drop=True, inplace=True)

df_concat.info()

In [ ]:
df_concat['type:TYPE'].value_counts()
df_concat['organ'].value_counts()

In [ ]:
df_concat=df_concat.replace('None', '_')
df_concat.info(); df_concat.head()

In [ ]:
df_concat['type:TYPE']=df_concat['type:TYPE'].apply(lambda x: x.upper())
df_concat.info(); df_concat.head()

In [ ]:
df_concat[df_concat['ref_count:int']==0]['type:TYPE'].value_counts()

In [ ]:
df_concat['type:TYPE'].value_counts()

In [ ]:
cols=list(df_directional.columns)
cols

In [ ]:
df_concat=df_concat[cols]
df_concat.head(5)

In [ ]:
%%time
df_concat.to_csv('./data/processed/relations.txt', sep='|', index=False, header=True)

In [ ]:
# list of column names for header file
cols=list(df_concat.columns)
cols

In [ ]:
# create headers file
df_rel_headers=pd.DataFrame(columns=cols)
df_rel_headers
df_rel_headers.to_csv('./data/processed/relations_header.txt', sep= '|', index=False)

## Nodes

In [ ]:
# define query
sql='select id, name, nodetype from resnet18.node where id is not null and name is not null and nodetype is not null'


In [ ]:
%%time

# save query results to file
with conn.cursor() as cur:
    with open('./data/raw/nodes_raw.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter='|')
        for record in cur.fetchall():
            line="|".join(map(str, record))+'\n'
            
            f.write(line)

### Create Node files

In [ ]:
df_node=pd.read_csv('./data/raw/nodes_raw.txt', delimiter='|', header=None, encoding='utf-8')

df_node.info(); df_node.head()

In [ ]:
# create column headers
df_node.columns=[':ID', 'name', ':LABEL']
df_node.head()

In [ ]:
# df cleanup
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.upper())
df_node[':LABEL']=df_node[':LABEL'].apply(lambda x: x.strip())
df_node['name']=df_node['name'].apply(lambda x: x.strip())
df_node[':ID']=df_node[':ID'].apply(lambda x: str(x).strip())  #convert to string and strip()
df_node[':ID']=df_node[':ID'].astype('int64')   # convert back to int
df_node.info(); df_node.head()

In [ ]:
# check for name strings that are too long
df_node['name_len']=df_node['name'].apply(lambda x: len(x))

In [ ]:
df_node.sort_values(by='name_len', ascending=False)

In [ ]:
# Does not seem to be an issue when preprocessing in a Linux environment
# Issue is leading quote with no ending quote in sql db for id -7235442027224814239 above
# the following is a temp fix unitil resolved in sql db
# process long name field

x=df_node[df_node[':ID']==-7235442027224814239]['name']
long_index=df_node[df_node[':ID']==-7235442027224814239].index[0] #capture index to delete later
long_index
x=x.to_frame().reset_index()
x.info(); x.head()
y=x.loc[0,'name']
len(y)

In [ ]:
# parse bad field
lst=y.split('\r\n')
lst=y.split('\n') # use this if above fails

lst[:5]
lst[0]='-7235442027224814239|' + lst[0]
lst[:5]

In [ ]:
# create list of lists from long name field
new_list=[]
for i in range(len(lst)):
    z=lst[i].split('|')
    new_list.append(z)
    

In [ ]:
len(new_list)
new_list[:5]

In [ ]:
# create df from list of lists
df_temp=pd.DataFrame(new_list, columns=[':ID', 'name', ':LABEL'])
df_temp.info(); df_temp.head()

In [ ]:
df_temp.dropna(inplace=True)
df_temp.info()

In [ ]:
df_temp[':LABEL']=df_temp[':LABEL'].apply(lambda x: x.upper())
df_temp.head()

In [ ]:
# combine dfs
df_node.drop(int(f'{long_index}'), inplace = True) #drop row with long name field
df_node.drop(columns=['name_len'], inplace=True)

df_node_new=pd.concat([df_node, df_temp])
df_node_new.reset_index(inplace=True, drop=True)
df_node_new.info(); df_node_new.head()

In [ ]:
# check for presence of ';;' & ';'
df_node_new[df_node_new['name'].str.contains(";")]
df_node_new[df_node_new['name'].str.contains(";;")]

In [ ]:
# convert single and double semi-colons to colons
df_node_new['name']=df_node_new['name'].str.replace(';;', ':')
df_node_new['name']=df_node_new['name'].str.replace(';', ':')


In [ ]:
df_node_new.to_csv('./data/processed/nodes.txt', sep='|', index=False, header=True)

In [ ]:
# create list of column names for header file
cols=list(df_node.columns)
cols

In [ ]:
# create and save header file
node_headers=pd.DataFrame(columns=cols)
node_headers
node_headers.to_csv('./data/processed/nodes_header.txt', sep='|', index=False)

In [ ]:
conn.close()

**To load data into Neo4j**, click on 'Add' in the project pane. Name the new project, click on the '...' by the 'Open' button for the project, and choose 'Terminal'. Take note of the dbms number at the cursor. Copy the nodes.txt, nodes_header.txt, relations.txt, and realations_header.txt files pepared above. Drop them in the import file found here: C:\Users\\[user_name]\\.Neo4jDesktop\relate-data\dbmss\dbms-##(number at cursor)\import

At the Neo4j terminal, change to the bin directory (type 'cd bin') and then paste the following at the cursor:
neo4j-admin import --delimiter="|" --nodes=import/nodes_header.txt, import/nodes.txt --relationships=import/relations_header.txt, import/relations.txt --skip-bad-relationships=true

Go back to the project pane, press 'Start'. The active database will now be shown in a pane at the top. Press open to use in the Neo4j browser.